### Инициализация класса

Создайте класс с именем MyTreeClf. Данный класс при инициализации должен принимать на вход следующие параметры:

max_depth – максимальная глубина.
По-умолчанию: 5
min_samples_split – кол-во объектов в листе, чтобы его можно было разбить и превратить в узел.
По-умолчанию: 2
max_leafs – максимальное количество листьев разрешенное для дерева.
По-умолчанию: 20
Все переданные (или дефолтные) параметры должны быть сохранены внутри класса.

При обращении к экземпляру класса (или при передачи его в функцию print) необходимо распечатать строку по следующему шаблону:

MyTreeClf class: max_depth=<max_depth>, min_samples_split=<min_samples_split>, max_leafs=<max_leafs>

In [ ]:
class MyTreeClf():
    def __init__(self, max_depth=5, min_samples_split=2, max_leafs=20):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.max_leafs = max_leafs

    def __str__(self):
        params = ', '.join(f'{key}={value}' for key, value in self.__dict__.items())
        return f'{__class__.__name__} class: {params}'

### Поиск наилучшего сплита

Т.к. код дерева довольно объемный, то, чтобы проще было его отладить, реализуем сначала поиск наилучшего сплита.

Напишите функцию get_best_split, которая делает следующее:

- Принимает на вход:
  - X – фичи в виде датафрейма пандаса
  - y – таргет в виде пандасовской серии
- Выполняет:
  - Проходится по всем колонкам датафрейма
  - Для каждой колонки формирует разделители для отсортированных уникальных значений
  - Для каждого разделителя вычисляет прирост информации
- Возвращает разделитель с наибольшим приростом информации:
  - col_name – название фичи, по которой лучше всего производить разделение
  - split_value – значение, по которому необходимо производить разделение
  - ig – прирост информации, который дает этот разделитель

Примечание #1

В процессе построения дерева может сложиться такая ситуация, что в одной из подвыборок (или даже в обоих) у вас будет только один класс. И вам нужно будет вычислить энтропию:



Если попытаетесь вычислить это напрямую с помощью NumPy, то получите ошибку.:

s = -(1*np.log2(1) + 0*np.log2(0))
Поскольку питон сначала попытается вычислить логарифм
n
p
.
l
o
g
2
(
0
)
np.log2(0), а это
−
i
n
f
−inf. Но это не важно, поскольку любое значение умноженное на 0 дает 0. А значит весь блок
0
⋅
n
p
.
l
o
g
2
(
0
)
=
0
0⋅np.log2(0)=0. Вам нужно будет как-то обыграть это в коде, чтобы не вычислять логарифм от 0.

Примечание #2

Для того чтобы подвыборки получались одинаковыми договоримся, что сравнивать разделитель с данными будем следующим образом:

левая
подвыборка
≤
разделитель
<
правая
подвыборка
леваяподвыборка≤разделитель<праваяподвыборка


In [ ]:
import pandas as pd
import numpy as np

def get_best_split(X: pd.DataFrame, y:pd.Series, min_samples_split: int=2):
    """
    Находит наилучший сплит в данных по всем признакам, основываясь на приросте информации.

    :param X: pandas.DataFrame - фичи
    :param y: pandas.Series - целевая переменная
    :param min_samples_split: int - минимальное количество объектов в группе после разделения
    :return: tuple (best_col_name, best_split_value, best_ig)
    """
    # Функция вычисление энтропии
    def calculate_entropy(y):
        if len(y) == 0:
            return 0
        else:
            counts = np.bincount(y)
            probabilities = counts / len(y)
            return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

    # инициализируем параметры для хранения информации лучшего разделителя
    best_ig = -1
    best_col_name = None
    best_split_value = None

    # Перебираем все колонки в датафрейме
    for col in X.columns:
        # Получаем уникальные отсортированные значения
        unique_values = np.sort(X[col].unique())

        # Генерируем разделителя (средние значения между соседними уникальными элементами)
        split_values = (unique_values[:-1] + unique_values[1:]) / 2

        for split_value in split_values:
            # разделяем данные на левую и правую группы
            left_mask = X[col] <= split_value
            right_mask = X[col] > split_value
            y_left, y_right = y[left_mask], y[right_mask]

            # пропускаем разделители, которые дают слишком малые группы
            if len(y_left) < min_samples_split or len(y_right) < min_samples_split:
                continue

            # вычисление энтропии
            base_entropy = calculate_entropy(y)    # изначальное состояние
            left_entropy = calculate_entropy(y_left)    # энтропия левой группы
            right_entropy = calculate_entropy(y_right)    # энтропия правой группы

            # вычисляем кол-во элементов групп для подсчета прироста информации
            n = len(y)    # общее кол-во элементов
            n_left = len(y_left)    # кол-во элементов левой группы
            n_right = len(y_right)    # кол-во элементов правой группы

            # вычисляем прирост информации
            ig = base_entropy - (n_left / n) * left_entropy - (n_right / n) * right_entropy

            # сохраняем лучший разделитель
            if ig > best_ig:
                best_ig = ig
                best_col_name = col
                best_split_value = split_value

    return best_col_name, best_split_value, best_ig

### Обучение

Пора учиться. Доработайте класс MyTreeClf следующим образом:

В инициализатор класса добавьте переменную leafs_cnt (не параметр, а именно переменную). В ней будет храниться количество листьев нашего дерева.
Добавьте в класс метод fit. Данный метод должен делать следующее:
На вход принимать две переменные:
- X — все фичи в виде датафрейма пандаса.
- y — целевая переменная в виде пандасовской серии.
Строить дерево по алгоритму, описанному на предыдущем шаге. Построенное дерево необходимо сохранить внутри экземпляра класса. Форму хранения иерархии выбирайте сами. Вариантов много: словари, вложенные списки, да хоть в текстовом виде — как вам удобно или как умеете.
При этом необходимо соблюдать заданные ограничения на максимальную глубину, максимальное кол-во листьев и допустимое кол-во экземпляров в узле.
Считать кол-во получившихся листьев в переменной leafs_cnt.
Метод ничего не возвращает.
Добавьте метод print_tree — данный метод должен отрисовывать дерево примерно такого вида:
```
variance > 0.320165
  skewness > 5.86535
    curtosis > 6.21865
      variance > -0.36205
        leaf_left = 1.0
        curtosis > 2.62465
          leaf_left = 1.0
          leaf_right = 0.0
      skewness > -4.6745
        variance > -0.357315
          leaf_left = 1.0
          leaf_right = 0.0
        variance > -2.1171
          leaf_left = 0.5
          leaf_right = 0.0
    variance > -3.4448999999999996
      leaf_left = 0.975
      leaf_right = 0.0
  leaf_right = 0.1076923076923077
```
Внешний вид никак проверяться не будет — он нужен вам для понимания и отладки.

На первый взгляд алгоритм дерева может показаться сложным, но это только первое впечатление. Он, наверное, самый объемный из всех базовых алгоритмов, но ничего сложного в нем нет. И за него дают целых 5 баллов :) Вот несколько советов, которые облегчат вам жизнь:

Попробуйте реализовать дерево в Jupyter Notebook. Разбейте код на отдельные функции (вычисление прироста, поиск лучшего сплита, итерация по фичам) и пишите каждую из них в отдельной ячейке, и тут же тестируйте. И, уже протестировав каждый компонент, собирайте все вместе.
Обязательно реализуйте вывод дерева — очень помогает понять, что происходит (например, вы вручную сможете посчитать количество листьев).
Old School: попробуйте расписать построение дерева на листе бумаге.
Если совсем ничего не получается, можете поизучать реализации в интернете. Может это натолкнет вас на интересные мысли.
Дополнительная проверка

К сожалению, ограничения платформы степик не позволяют провести все необходимые проверки для дерева. Поэтому в некоторых ситуациях тестовая система может принять код, который на самом деле еще не полностью отлажен.

Чтобы этого избежать я сгенерировал 6 деревьев на датасете Banknote Authentication (см. введение) и привел ниже их параметры и характеристики. Можете использовать их для локальной проверки. Для начала сверьте сходится ли количество листьев и сумма их значений. Если где-то есть расхождения, копайте глубже - сверьте распечатки деревьев и попробуйте в ручном режиме посчитать в каком месте расходится и понять почему.